<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [4]:
import requests

# 文件下载链接
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip'
# 目标文件名
file_path = 'concrete_data_week4.zip'

# 发送请求并保存响应内容
response = requests.get(url)
if response.status_code == 200:
    with open(file_path, 'wb') as f:
        f.write(response.content)
    print("文件下载成功")
else:
    print("文件下载失败，状态码：", response.status_code)


文件下载成功


In [5]:
import zipfile

# 指定要解压的文件和目标路径
with zipfile.ZipFile('concrete_data_week4.zip', 'r') as zip_ref:
    zip_ref.extractall('concrete_data_week4')
    print("文件解压成功")


文件解压成功


In [10]:
# 更新 base_directory 来指向正确的路径
base_directory = 'concrete_data_week4/concrete_data_week4'

# 重新检查 train 和 valid 文件夹的内容
train_directory = os.path.join(base_directory, 'train')
valid_directory = os.path.join(base_directory, 'valid')

# 打印更新后的目录内容
try:
    print("Contents of train directory:", os.listdir(train_directory))
    print("Contents of valid directory:", os.listdir(valid_directory))
except Exception as e:
    print("Error accessing directories:", e)
    # 如果还是有错误，打印整个 base_directory 的内容看看具体结构
    print("Contents of base_directory:", os.listdir(base_directory))


Contents of train directory: ['.DS_Store', 'negative', 'positive']
Contents of valid directory: ['.DS_Store', 'negative', 'positive']


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

# 训练数据的数据增强
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 验证数据没有数据增强，只应用预处理
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [16]:
train_generator = train_datagen.flow_from_directory(
    train_directory,  # 正确的训练数据路径
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    valid_directory,  # 正确的验证数据路径
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [17]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# 加载VGG16预训练模型，不包括顶层
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 冻结VGG16的所有层，以防止在训练过程中更新权重
for layer in base_model.layers:
    layer.trainable = False

# 创建新的顺序模型
model = Sequential([
    base_model,  # 添加预训练的底层
    Flatten(),   # 扁平化输出以便添加全连接层
    Dense(512, activation='relu'),  # 全连接层
    Dropout(0.5),  # 添加Dropout防止过拟合
    Dense(train_generator.num_classes, activation='softmax')  # 输出层，类别数与训练数据生成器中的类别数相匹配
])

# 检查模型结构
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ ?                           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [19]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),  # 使用 learning_rate 设置学习率
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [21]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5  # 或者根据需要调整epochs数量
)


Epoch 1/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 2060s 7s/step - accuracy: 0.9898 - loss: 0.0542 - val_accuracy: 0.9969 - val_loss: 0.0135
Epoch 2/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 555us/step - accuracy: 0.9800 - loss: 0.0561 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 2318s 8s/step - accuracy: 0.9924 - loss: 0.0416 - val_accuracy: 0.9965 - val_loss: 0.0111
Epoch 4/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 482us/step - accuracy: 0.9900 - loss: 0.0100 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
300/300 ━━━━━━━━━━━━━━━━━━━━ 2261s 8s/step - accuracy: 0.9937 - loss: 0.0282 - val_accuracy: 0.9969 - val_loss: 0.0113


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).